<a href="https://colab.research.google.com/github/lkozina1309/tensorflow/blob/main/CNN/Jupyter/data_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Script *data_augmentation.py* adds data_augmentation to classification of dogs and cats to bring more accuracy to the model. 

First, we have to import the libraries.

In [ ]:
import os
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.models import Sequential
import tensorflow_datasets as tfds

Then, we have to download the dataset.


In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path_to_zip = tf.keras.utils.get_file('cats_and_dogs.zip', origin=_URL, extract=True)
PATH = os.path.join(os.path.dirname(path_to_zip), 'cats_and_dogs_filtered')
train_dir = os.path.join(PATH, 'train')
BATCH_SIZE = 64
IMG_SIZE = (32, 32)

train_dataset = tf.keras.utils.image_dataset_from_directory(train_dir,
                                                            shuffle=True,
                                                            batch_size=BATCH_SIZE,
                                                            image_size=IMG_SIZE)

We can check our classes.

In [ ]:
class_names = train_dataset.class_names

print(class_names)

Now, we'll use validation split and create test set as original dataset doesn't contain it.

In [ ]:
validation_dir = os.path.join(PATH, 'validation')

validation_dataset = tf.keras.utils.image_dataset_from_directory(validation_dir,
                                                                 shuffle=True,
                                                                 batch_size=BATCH_SIZE,
                                                                 image_size=IMG_SIZE)

Configure the dataset.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_dataset = train_dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

We'll use data rescaling as well as random flipping and rotation.

In [ ]:

rescale = tf.keras.Sequential([
  layers.Rescaling(1./255)
)
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

Our model is going to have 3 convolutional layers and 2 max_pooling layers beetween and we'll add dropout to avoid overfitting.

In [ ]:
model = tf.keras.Sequential([
resize_and_rescale,
data_augmentation,
layers.Conv2D(16, (3, 3), activation='relu', input_shape=(32, 32, 3)),
layers.MaxPooling2D(2, 2),
layers.Conv2D(32, (3, 3), activation='relu'),
layers.MaxPooling2D(2, 2),
layers.Conv2D(64, (3, 3), activation='relu'),
layers.Dropout(0.4),
layers.Flatten(),
layers.Dense(128, activation='relu'),
layers.Dense(2),
])

We'll use "adam" as an optimizer and "SparseCategoricalCrossentropy" as a loss function and we'll run it through 10 epochs.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_dataset,
                    epochs=30, 
                    validation_data=validation_dataset)

We should get accuracy of more than 70%.

In [ ]:
test_loss, test_acc = model.evaluate(test_dataset, verbose=2)
print(test_acc)